## 개요
PyTorch를 사용하여 장치 간의 모델을 저장하거나 불러오는 것은 비교적 간단하다. 이번 레시피에서는, CPU와 GPU에서 모델을 저장하고 불러오는 방법을 실험한다.

## 단계
1. 데이터 활용에 필요한 모든 라이브러리 import
2. 신경망을 구성하고 초기화
3. GPU에서 저장하고 CPU에서 불러오기
4. GPU에서 저장하고 GPU에서 불러오기
5. CPU에서 저장하고 GPU에서 불러오기
6. ```DataParallel```모델을 저장하고 불러오기

### 1. 데이터 활용에 필요한 모든 라이브러리 import

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim

### 2. 신경망을 구성하고 초기화

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### 3. GPU에서 저장하고 CPU에서 불러오기
GPU에서 학습된 모델을 CPU에서 불러올 때는 ```torch.load()``` 함수의 ```map_location```인자에 ```torch.device('cpu')```를 전달한다.

In [3]:
PATH = "model.pt"

torch.save(net.state_dict(), PATH)

device = torch.device('cpu')
model = Net()
model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

Tensor의 저장된 내용은 ```map_location``` 인자를 통하여 CPU 장치에 동적으로 재배치된다.

### 4. GPU에서 저장하고 GPU에서 불러오기
GPU에서 학습하고 저장된 모델을 GPU에서 불러올 때는, 초기화된 모델에 ```model.to(torch.device('cuda'))```을 호출하여 CUDA에 최적환 모델로 변환해준다. <br>
모든 입력에 ```.to(torch.device('cuda'))``` 함수를 호출해야 모델에 데이터를 제공할 수 있다.

In [4]:
# 저장하기
torch.save(net.state_dict(), PATH)

# 불러오기
device = torch.device("cuda")
model = Net()
model.load_state_dict(torch.load(PATH))
model.to(device)

AssertionError: Torch not compiled with CUDA enabled

### 5. CPU에서 저장하고 GPU에서 불러오기
cpu에서 학습하고 저장된 모델을 GPU에서 불러올 때는, ```torch.load()```함수의 ```map_location```인자를 ```cuda:device_id```로 설정. 그러면 주언 GPU 장치에서 모델이 불러와진다.
<br>
모델의 매개변수 Tensor를 CUDA Tensor로 변환하기 위해, ```model.to(torch.device('cuda'))```를 호출

### 6. ```torch.nn.DataParallel``` 모델을 저장하고 불러오기
```torch.nn.DataParallel```은 병렬 GPU 활용을 가능하게 하는 모델 wrapper이다. 그러면 원하는 장치에 원하는 방식으로 유연하게 모델을 불러올 수 있다.

In [6]:
# 저장하기
torch.save(net.module.state_dict(), PATH)

# 사용할 장치에 불러오기

AttributeError: 'Net' object has no attribute 'module'